In [1]:
import os, sys, glob, re, math, pickle
import scprep, magic, phate
from scipy import sparse
import numpy as np
import pandas as pd
from scipy import sparse as sp
import time,random,datetime
import scanpy as sc
#from bbknn import bbknn
import anndata
from typing import Dict, Optional
import tables
from bbknn import bbknn
%matplotlib inline
%load_ext memory_profiler
import graphtools as gt

# fps
dfp = '/home/cl2292/project/SBMA/data/'
pfp = '/home/cl2292/project/SBMA/result/'
pdfp = '/home/cl2292/project/SBMA/data/processed/'
sc.settings.figdir = pfp

In [2]:
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230201_ctype.h5ad' # for full, can maybe get away with ~300G
    %memit adata = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))

peak memory: 2619.00 MiB, increment: 2349.28 MiB
loaded @230201.10:32:34
took 3.19-s to load data


In [3]:
#Imputation

wt = adata[adata.obs['genotype']=='WT', :]
mut = adata[adata.obs['genotype']=='AR', :]

In [4]:
#k=45, t=3
print('Starting imputation for {}\n'.format('WT'))
tic = time.time()

wt = adata[adata.obs['genotype']=='WT', :]
wt.obs['value'] = 0
sc.pp.pca(wt)
sc.pp.neighbors(wt, n_pcs=45)


# MAGIC
G = gt.Graph(data=wt.obsp['connectivities']+sparse.diags([1]*wt.shape[0],format='csr'),
             precomputed='adjacency',
             use_pygsp=True)
G.knn_max = None

magic_op=magic.MAGIC().fit(X=wt.X,graph=G) # running fit_transform produces wrong shape
wt.layers['imputed']=magic_op.transform(wt.X,genes='all_genes')

print('\n  imputation in {:.2f}-min'.format((time.time() - tic)/60))


# save data objects
wt.write(os.path.join(pdfp,'230201_WT_imp.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))




print('\n Starting imputation for {}\n'.format('SBMA'))
tic = time.time()

mut = adata[adata.obs['genotype']=='AR', :]
mut.obs['value'] = 0
sc.pp.pca(mut)
sc.pp.neighbors(mut, n_pcs=45)

# MAGIC
G = gt.Graph(data=mut.obsp['connectivities']+sparse.diags([1]*mut.shape[0],format='csr'),
             precomputed='adjacency',
             use_pygsp=True)
G.knn_max = None

magic_op=magic.MAGIC().fit(X=mut.X,graph=G) # running fit_transform produces wrong shape
mut.layers['imputed']=magic_op.transform(mut.X,genes='all_genes')

print('\n  imputation in {:.2f}-min'.format((time.time() - tic)/60))

# save data objects
mut.write(os.path.join(pdfp,'230201_SBMA_imp.h5ad'))
print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

Starting imputation for WT



/tmp/ipykernel_9256/3770459987.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  wt.obs['value'] = 0


Running MAGIC on 28501 cells and 27091 genes.
Using precomputed graph and diffusion operator...
Running MAGIC with `solver='exact'` on 27091-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.


/gpfs/ycga/project/lim_janghoo/cl2292/conda_envs/af/lib/python3.10/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(
/gpfs/ycga/project/lim_janghoo/cl2292/conda_envs/af/lib/python3.10/site-packages/magic/magic.py:541: UserWarning: Running MAGIC.transform on different data to that which was used for MAGIC.fit may not produce sensible output, unless it comes from the same manifold.
  warnings.warn(


Calculating imputation...
Calculated imputation in 67.34 seconds.

  imputation in 1.90-min
saved @230201.10:34:35

 Starting imputation for SBMA



/tmp/ipykernel_9256/3770459987.py:34: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  mut.obs['value'] = 0


Running MAGIC on 30222 cells and 27091 genes.
Using precomputed graph and diffusion operator...
Running MAGIC with `solver='exact'` on 27091-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.


/gpfs/ycga/project/lim_janghoo/cl2292/conda_envs/af/lib/python3.10/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(
/gpfs/ycga/project/lim_janghoo/cl2292/conda_envs/af/lib/python3.10/site-packages/magic/magic.py:541: UserWarning: Running MAGIC.transform on different data to that which was used for MAGIC.fit may not produce sensible output, unless it comes from the same manifold.
  warnings.warn(


Calculating imputation...
Calculated imputation in 73.37 seconds.

  imputation in 1.80-min
saved @230201.10:36:30
